In [3]:
import pathlib
from pprint import pprint
from matplotlib import pyplot as plt
import numpy as np
import itertools
from lips.benchmark.powergridBenchmark import PowerGridBenchmark
from lips.utils import get_path

In [29]:
# indicate required paths
LIPS_PATH = pathlib.Path().resolve().parent # it is supposed that the notebook had run from getting_started folder
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_case14_sandbox"
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "benchmarks" / "l2rpn_case14_sandbox.ini"
SIM_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "simulators"
BASELINES_PATH = LIPS_PATH / "trained_baselines" / "powergrid"
TRAINED_MODEL_PATH = LIPS_PATH / "trained_models" / "powergrid"
EVALUATION_PATH = LIPS_PATH / "evaluation_results" / "PowerGrid"
LOG_PATH = LIPS_PATH / "lips_logs.log"

## Benchmark1

In [30]:
benchmark1 = PowerGridBenchmark(benchmark_name="Benchmark1",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

In [ ]:
benchmark1._fills_actor_simulator()
env = benchmark1.env

### Verify that the number of line disconnections are respected for all the datasets

In [32]:
# there should be zero disconnection for more than two lines in training data
line_status_train = benchmark1.train_dataset.data["line_status"]
assert np.sum(env.n_line - np.sum(line_status_train, axis=1) > 2) == 0
# there should be zero disconnection for more than two lines in test data
line_status_test = benchmark1._test_dataset.data["line_status"]
assert np.sum(env.n_line - np.sum(line_status_test, axis=1) > 2) == 0
# there should be zero disconnection for more than three lines in ood test data
line_status_test_ood = benchmark1._test_ood_topo_dataset.data["line_status"]
assert np.sum(env.n_line - np.sum(line_status_test_ood, axis=1) > 3) == 0

### Verify that the topology changes are not applied on more than 3 substations

In [33]:
topo_vect = benchmark1.train_dataset.data["topo_vect"]
sub_topo = np.array(list(itertools.chain.from_iterable(itertools.repeat(x, l) 
                                                       for x, l in zip(np.arange(env.n_sub),env.sub_info))))

In [34]:
changed_subs = []
for sub_id in range(env.n_sub):
    changed_subs.append(np.sum(topo_vect[:, sub_topo==sub_id] == 2, axis=1)>0)
changed_subs = np.vstack(changed_subs)

In [35]:
assert sum(np.sum(changed_subs, axis=0) > 3) == 0

##  Benchmark2

In [61]:
benchmark2 = PowerGridBenchmark(benchmark_name="Benchmark2",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=CONFIG_PATH
                               )

In [ ]:
benchmark2._fills_actor_simulator()
env = benchmark2.env

### verify that the number of line disconnections is respected for all the datasets

In [65]:
# there should be zero disconnection for more than two lines in training data
line_status_train = benchmark2.train_dataset.data["line_status"]
assert np.sum(env.n_line - np.sum(line_status_train, axis=1) > 2) == 0
# there should be zero disconnection for more than two lines in test data
line_status_test = benchmark2._test_dataset.data["line_status"]
assert np.sum(env.n_line - np.sum(line_status_test, axis=1) > 2) == 0
# there should be zero disconnection for more than three lines in ood test data
line_status_test_ood = benchmark2._test_ood_topo_dataset.data["line_status"]
assert np.sum(env.n_line - np.sum(line_status_test_ood, axis=1) > 2) == 0

### Verify that there at most one sub change during training

In [67]:
topo_vect = benchmark2.train_dataset.data["topo_vect"]
sub_topo = np.array(list(itertools.chain.from_iterable(itertools.repeat(x, l) 
                                                       for x, l in zip(np.arange(env.n_sub),env.sub_info))))
changed_subs = []
for sub_id in range(env.n_sub):
    changed_subs.append(np.sum(topo_vect[:, sub_topo==sub_id] == 2, axis=1)>0)
changed_subs = np.vstack(changed_subs)

In [68]:
changed_subs

array([[False, False, False, ..., False, False, False],
       [ True, False,  True, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [70]:
assert sum(np.sum(changed_subs, axis=0) > 1) == 0

### Verify that there are always one sub change during test

In [71]:
topo_vect = benchmark2._test_dataset.data["topo_vect"]
sub_topo = np.array(list(itertools.chain.from_iterable(itertools.repeat(x, l) 
                                                       for x, l in zip(np.arange(env.n_sub),env.sub_info))))
changed_subs = []
for sub_id in range(env.n_sub):
    changed_subs.append(np.sum(topo_vect[:, sub_topo==sub_id] == 2, axis=1)>0)
changed_subs = np.vstack(changed_subs)

In [80]:
np.where(np.sum(changed_subs, axis=0) != 1)

(array([1253, 4334, 4523, 4750, 6721, 9214, 9224]),)

In [89]:
# this does not work
assert sum(np.sum(changed_subs, axis=0) != 1) == 0

AssertionError: 

In [92]:
tolerance = 20 # until 20 observations could be for any reason not respect exactly the required behavior
assert (tolerance - sum(np.sum(changed_subs, axis=0) != 1)) > 0 

### Verify that there are always two sub changes during test ood

In [93]:
topo_vect = benchmark2._test_ood_topo_dataset.data["topo_vect"]
sub_topo = np.array(list(itertools.chain.from_iterable(itertools.repeat(x, l) 
                                                       for x, l in zip(np.arange(env.n_sub),env.sub_info))))
changed_subs = []
for sub_id in range(env.n_sub):
    changed_subs.append(np.sum(topo_vect[:, sub_topo==sub_id] == 2, axis=1)>0)
changed_subs = np.vstack(changed_subs)

In [94]:
assert sum(np.sum(changed_subs, axis=0) != 2) == 0

AssertionError: 

In [95]:
np.where(np.sum(changed_subs, axis=0) != 2)

(array([ 217, 2131, 4155, 4635, 4727, 6458, 6935, 8674, 8817]),)

In [99]:
tolerance = 20 # until 20 observations could be for any reason not respect exactly the required behavior
assert (tolerance - sum(np.sum(changed_subs, axis=0) != 2)) > 0 

## Benchmark3

In [ ]:
benchmark3 = PowerGridBenchmark(benchmark_name="Benchmark3",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=CONFIG_PATH
                               )

# NIPS Track1 environment

In [39]:
LIPS_PATH = pathlib.Path().resolve().parent
BENCH_CONFIG_PATH = LIPS_PATH / "configurations" / "powergrid" / "l2rpn_neurips_2020_track1_small.ini"
DATA_PATH = LIPS_PATH / "reference_data" / "powergrid" / "l2rpn_neurips_2020_track1_small"
LOG_PATH = LIPS_PATH / "lips_logs_neurips.log"

In [40]:
benchmark1 = PowerGridBenchmark(benchmark_name="Benchmark1",
                                benchmark_path=DATA_PATH,
                                load_data_set=True,
                                log_path=LOG_PATH,
                                config_path=BENCH_CONFIG_PATH
                               )

In [ ]:
benchmark1._fills_actor_simulator()
env = benchmark1.env

## Benchmark1

### Verify that the number of line disconnections are respected for all the datasets

In [42]:
# there should be zero disconnection for more than two lines in training data
line_status_train = benchmark1.train_dataset.data["line_status"]
assert np.sum(env.n_line - np.sum(line_status_train, axis=1) > 2) == 0
# there should be zero disconnection for more than two lines in test data
line_status_test = benchmark1._test_dataset.data["line_status"]
assert np.sum(env.n_line - np.sum(line_status_test, axis=1) > 2) == 0
# there should be zero disconnection for more than three lines in ood test data
line_status_test_ood = benchmark1._test_ood_topo_dataset.data["line_status"]
assert np.sum(env.n_line - np.sum(line_status_test_ood, axis=1) > 3) == 0

### Verify that the topology changes are not applied on more than 2 substations

In [44]:
topo_vect = benchmark1.train_dataset.data["topo_vect"]
sub_topo = np.array(list(itertools.chain.from_iterable(itertools.repeat(x, l) 
                                                       for x, l in zip(np.arange(env.n_sub),env.sub_info))))

In [47]:
changed_subs = []
for sub_id in range(env.n_sub):
    changed_subs.append(np.sum(topo_vect[:, sub_topo==sub_id] == 2, axis=1)>0)
changed_subs = np.vstack(changed_subs)

In [48]:
assert sum(np.sum(changed_subs, axis=0) > 2) == 0